In [1]:
import os
import tensorflow as tf
import pandas as pd

tf.__version__


'2.13.0'

In [2]:
main_dir = r"F:\linux_ubuntu_codes\codes\nlp_lecture"

In [3]:
train_dir = os.path.join(main_dir + r"\train.csv")
test_dir = os.path.join(main_dir + r"\test.csv")

'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'

In [4]:
train_dir

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Make a train_df
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 1)
train_df_shuffled.head()

(7613, 5)

In [7]:
train_df_shuffled.shape

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
# How many examples of each class?
train_df['target'].value_counts()

(7613, 3263)

In [9]:
# How many total samples?
len(train_df), len(test_df)

Target:1 (real disaster)
Text: 
@NBCNews Yea bombing #pearlharbor not so good of an idea!
-----

Target:0 (not a real disaster)
Text: 
@kyrikoni @ExpressandStar Who said veg and fruit was good for you. Hope there's been no injuries or fatalities.
-----

Target:0 (not a real disaster)
Text: 
https://t.co/4i0rKcbK1D
SON OF SAVIOR LAVA VIDEO
-----

Target:0 (not a real disaster)
Text: 
Guns are for protection.. 
That shit really shouldn't be used unless your life in danger
-----

Target:1 (real disaster)
Text: 
Young dancer moves about 300 youth in attendance at the GMMBC Youth Explosion this past Saturday. Inspiring! http://t.co/TMmOrvxsWz
-----


In [ ]:
# Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index +5].itertuples():
    _, text, target = row
    if target > 0:
        print(f"Target:{target}", "(real disaster)")
    else:
        print(f"Target:{target}", "(not a real disaster)")
    
    print(f"Text: \n{text}")
    print("-----\n")

### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'],
                                                                            train_df['target'],
                                                                            test_size = 0.1,
                                                                            random_state = 42)

(7613,)

In [12]:
train_df['text'].shape

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [13]:
train_df["text"]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [14]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
train_df['text'].to_numpy()

(6851, 762, 6851, 762)

In [ ]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

### Converting text into numbers: Text Vectorization also known as Tokenization

In [17]:
# Get the total words
total_words = 0
for i in train_sentences:
    total_words += len(i.split())

total_words

(6851,)

In [18]:
train_sentences.shape

15

In [19]:
# Get the average words per sentence or line
avg_words_per_sentence = round(total_words/len(train_sentences))
avg_words_per_sentence

In [20]:
# Normally, total number of words is equal or less than the max vocabulary length
max_vocab_length = 10000

# max length of a statement is equal or more than the average word per sentence or line
max_length = 15

In [21]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

There's a flood in my street! Sentence length is 29.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[282,   3, 206,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [22]:
# Create a sample sentence and tokenize it
samp_sentence = "There's a flood in my street!"
print(samp_sentence, f"Sentence length is {len(samp_sentence)}.")
text_vectorizer([samp_sentence])
#len(tf.squeeze(text_vectorizer([samp_sentence])))

Original text: 

 Junko was 13 years old when the atomic bomb exploded over #Hiroshima on August 6 1945. http://t.co/lSpnyCVoLO http://t.co/Nh5pkFBfqm 

Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,  23, 868, 144, 194,  45,   2, 214, 115, 422,  60, 128,  11,
        469, 448]], dtype=int64)>

In [23]:
# Choose random sentence from the training dataset and tokenize it
rand_train_sentence = random.choice(train_sentences)
print(f"Original text: \n\n {rand_train_sentence} \n\nVectorized text:")
text_vectorizer([rand_train_sentence])

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'are',
 'be',
 'was',
 'have',
 'like',
 'as',
 'me',
 'but',
 'up',
 'just',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'all',
 'after',
 'its',
 'has',
 'no',
 'will',
 'an',
 'fire',
 'when',
 'if',
 'we',
 'get',
 'now',
 'more',
 'via',
 'new',
 'about',
 'what',
 'people',
 'or',
 'news',
 'he',
 'they',
 'over',
 'one',
 'been',
 'how',
 'dont',
 'who',
 'video',
 'into',
 'were',
 'do',
 'us',
 'can',
 'emergency',
 '2',
 'disaster',
 'there',
 'her',
 'some',
 'than',
 'still',
 'would',
 'his',
 'police',
 'crash',
 'burning',
 'suicide',
 'body',
 'california',
 'back',
 'time',
 'buildings',
 'had',
 'why',
 'off',
 'them',
 'got',
 'man',
 'see',
 'storm',
 'know',
 'going',
 'cant',
 'world',
 'first',
 'day',
 'rt',
 'nuclear',
 'love',
 'youtube',
 'our',
 'attack',
 'go',
 'fires',
 'two',
 'their',
 'bomb',

In [24]:
# Get unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab

10000

In [25]:
len(words_in_vocab)

['', '[UNK]', 'the', 'a', 'in']

In [26]:
# Top 5 words in words_in_vocab
words_in_vocab[:5]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

In [ ]:
# Least 5 words in words_in_vocab
words_in_vocab[-5:]

### Embedding layer

To make our embedding, we are going to use TensorFlow’s embedding layer: https://www.tensorflow.org/api/docs/python/tf/keras/layers/

The parameters are below mostly used.
* input_dim = size of the vocabulary
* output_dim = size of the output embedding vector.  For example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 64,
                             embeddings_initializer = "uniform",
                             input_length = max_length)

Original text: Injuries Illnesses and Fatalities Latest Numbers : http://t.co/1uo1aTrbbJ

Vectorized text: [[ 289    1    7  362  199 2839    1    0    0    0    0    0    0    0
     0]]

Vectorized text shape: (1, 15)
Embbedded text: [[[-0.02796229  0.00588357  0.02138958 -0.0025916   0.04152458
    0.01366184 -0.04125395  0.03566234 -0.02768184 -0.0024143
   -0.0013608   0.02672127 -0.0143005  -0.01799623  0.03869674
    0.02519162 -0.01105823 -0.04477582 -0.04683232  0.04672701
    0.0125914  -0.01097994  0.04220506 -0.01432807 -0.0268524
   -0.03385969  0.0295727  -0.02836042 -0.02037652 -0.04936385
    0.00719299  0.03403104 -0.0408483   0.04331813 -0.01667015
   -0.02885242  0.04565011 -0.03498325  0.03235954  0.01700355
   -0.03073397 -0.03283228  0.00083069 -0.01012558 -0.00157117
    0.01312574 -0.02744392  0.03897801 -0.01690521 -0.00734366
   -0.0432062  -0.01817285  0.03904152 -0.00651119 -0.02562679
   -0.04284923 -0.00742116  0.01538116 -0.00549706 -0.03091816
    0.0050

In [ ]:
# Get a random sentence from the training set and return Original text, Vectorized text, Embedded text
rand_sentence = random.choice(train_sentences)
print(f"Original text: {rand_sentence}\n")
print(f"Vectorized text: {text_vectorizer([rand_sentence])}\n")
print(f"Vectorized text shape: {text_vectorizer([rand_sentence]).shape}")
print(f"Embbedded text: {embedding(text_vectorizer([rand_sentence]))}")
print(f"Embedded text shape: {embedding(text_vectorizer([rand_sentence])).shape}")

### Model_4: Bidirectional-LSTM Model (RNN)

In [30]:
# Build a Bidirectional RNN using tf
from tensorflow.keras import layers
inputs = layers.Input(shape= (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_bidirectional")

# Compile the model
model_4.compile(loss = 'binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 15, 64)            640000    
                                                                 
 bidirectional (Bidirection  (None, 15, 128)           66048     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                             

In [31]:
model_4.summary()

Epoch 1/5
215/215 [==============================] - 7s 14ms/step - loss: 0.5085 - accuracy: 0.7546 - val_loss: 0.4735 - val_accuracy: 0.7927
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3077 - accuracy: 0.8697 - val_loss: 0.5143 - val_accuracy: 0.7585
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2014 - accuracy: 0.9275 - val_loss: 0.5948 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1327 - accuracy: 0.9539 - val_loss: 0.7216 - val_accuracy: 0.7520
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0978 - accuracy: 0.9654 - val_loss: 0.8826 - val_accuracy: 0.7717


In [32]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels))

24/24 [==============================] - 1s 3ms/step


array([[2.90588941e-03],
       [4.39843908e-02],
       [2.83487421e-03],
       [1.15216209e-03],
       [1.25471652e-01],
       [4.79591358e-03],
       [4.84226941e-04],
       [1.48925129e-02],
       [1.15532734e-01],
       [9.92810667e-01],
       [7.62113154e-01],
       [3.58268082e-01],
       [2.44195410e-03],
       [5.65950535e-02],
       [3.79291177e-02],
       [5.49180470e-02],
       [9.96235788e-01],
       [6.42531412e-03],
       [9.99671817e-01],
       [3.48502677e-03],
       [1.17237449e-01],
       [8.05191875e-01],
       [4.75273430e-02],
       [9.34544027e-01],
       [1.18779503e-02],
       [4.95914035e-02],
       [9.97384489e-01],
       [9.49508905e-01],
       [9.99162853e-01],
       [5.06573182e-04],
       [1.55959632e-02],
       [2.10690987e-03],
       [7.31477737e-01],
       [3.32707077e-01],
       [1.00343581e-02],
       [2.29298800e-01],
       [1.72012709e-02],
       [3.58703807e-02],
       [2.35470780e-03],
       [4.87792939e-02],


In [33]:
# Make predictions with our bidirectional model
model_4_preds_probs = model_4.predict(val_sentences)
model_4_preds_probs

NameError: name 'model_4_pred_probs' is not defined

In [ ]:
model_4_pred_probs.shape

In [ ]:
tf.squeeze(model_4_pred_probs)

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds

In [ ]:
val_labels.shape

In [ ]:
# Evaluate the model by comparing the model_4_preds to val_labels
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_4_accuracy_score = accuracy_score(val_labels, model_4_preds)
model_4_precision, model_4_recall, model_4_fscore, _ =precision_recall_fscore_support(val_labels,
                                                                                      model_4_preds, average="weighted")

In [ ]:
dict_result = {"model_4_accuracy_score": model_4_accuracy_score,
               "model_4_precision": model_4_precision,
               "model_4_recall": {model_4_recall},
               "model_4_f1score": {model_4_fscore}}
dict_result